In [1]:
import json
import pandas as pd
from pandas import DataFrame
import numpy as np

In [2]:
#loading the  dataset
with open ("C:/Users/Aakash Sadaphule/Desktop/Untitled Folder/archive/MMHS150K_GT.json") as f:
    data = json.load(f)

In [3]:
data=pd.DataFrame.from_dict(data)
data=data.T
data=data.reset_index()
data.rename(columns={"index":"user_id"},inplace=True)

In [4]:
print('No. of rows: {} and columns are: {}'.format(data.shape[0],data.shape[1]))
data.head()

No. of rows: 149823 and columns are: 6


,user_id,img_url,labels,tweet_url,tweet_text,labels_str
0,1114679353714016256,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,"[Religion, Racist, Homophobe]"
1,1063020048816660480,http://pbs.twimg.com/ext_tw_video_thumb/106301...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,"[OtherHate, OtherHate, OtherHate]"
2,1108927368075374593,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,"[NotHate, NotHate, NotHate]"
3,1114558534635618305,http://pbs.twimg.com/ext_tw_video_thumb/111401...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[Racist, NotHate, NotHate]"
4,1035252480215592966,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,"[Racist, NotHate, Racist]"


In [5]:
#importing packages for data preporcessing
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
    
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
import re

[nltk_data] Downloading package stopwords to C:\Users\Aakash
[nltk_data]     Sadaphule\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to C:\Users\Aakash
[nltk_data]     Sadaphule\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Aakash
[nltk_data]     Sadaphule\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data preporcessing

In [6]:
def preprocessing(data):
    stemmer = nltk.stem.RSLPStemmer()
    stop_words = stopwords.words('english')
    stop_words.remove('not')
    text_data = []
    for tweet in data:
      review = re.sub(r"@[A-Za-z0-9_]+", " ", tweet)
      review = re.sub('RT', ' ', review)
      review = re.sub(r"https?://[A-Za-z0-9./]+", " ", review)
      review = re.sub(r"https?", " ", review)
      review = re.sub('[^a-zA-Z]', ' ', review)
      review = review.lower()
      review = review.split()
      porter_stemmer = PorterStemmer()
      review = [porter_stemmer.stem(word) for word in review if not word in set(stop_words) if len(word) > 2]
      review = ' '.join(review)
      text_data.append(review)

    return np.array(text_data)

In [7]:
text_data = preprocessing(data['tweet_text'])
text_data

array(['nigga', 'hors retard',
       'nigga momma youngboy spit real shit nigga', ...,
       'nigga big shitti',
       'say nigga rich amp said anger mmph lhhni lhhnyreunion',
       'nigga joe budden said thano got galact ass'], dtype='<U110')

# Word2vec model

In [8]:
import os
from os.path import join
dataset_directory="C:/Users/Aakash Sadaphule/Desktop/Untitled Folder/archive"
with open(join(dataset_directory,'MMHS150K_GT.json')) as json_file:
    HS_data = json.load(json_file)
hate_data = HS_data.copy()

In [9]:
import operator
def get_max_item(a_dict):
  return max(a_dict.items(), key=operator.itemgetter(1))[0]
get_max_item({'Religion': 1, 'Racist': 1})

'Religion'

In [10]:
def labels_to_single(labels):
  label_dict={}
  for l in labels:
    if l not in label_dict:
      label_dict[l] = 1
    else:
      label_dict[l] += 1
  return get_max_item(label_dict)

In [11]:
hs_data = [[key, val['tweet_text'], val['labels_str'], labels_to_single(val['labels_str'])] 
              # 1 if sum(to_label(val['labels'])) == 3 else 0] 
             for key, val in hate_data.items()]

In [12]:
from sklearn.model_selection import train_test_split
X = [row[1] for row in hs_data]
y = [row[3] for row in hs_data]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Word2Vec

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [14]:
punctuations = "?:!.,;[]-"
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
def tokenize_and_remove_stop_words(text):
    #word_tokenizer = NLTKWordTokenizer()
    # inputs = ['']

    raw_tokens = word_tokenize(text)
    final_tokens = [token.lower() for token in raw_tokens
            if token not in punctuations and token not in stop_words]
    # print(final_tokens)
    return final_tokens


def lemmatize(words):
    return [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]

def to_word2vec_vector(sent_items):
    model = Word2Vec(sent_items, min_count=1, vector_size=50, workers=3, window=3, sg=1)
    return model

def get_sentence_vectors(word2vec_model, sentence_tokens):
    res = []
    for sentence in sentence_tokens:  # TODO: fix two loops
        sentence_vectors = np.array([word2vec_model.wv[token] for token in sentence])
        mean = np.mean(sentence_vectors, axis=0)
        res.append(mean)

    return np.array(res)

def vectorize_by_wor2vec(text_instances):
  tokenized_items = []
  for ti in text_instances:
      tokens = tokenize_and_remove_stop_words(ti)
      tokens = lemmatize(tokens)
      # print(tokens)
      tokenized_items.append(tokens)

  #print(tokenized_items)
  model = to_word2vec_vector(tokenized_items)
  print(f'shape of word2vec model (vocabulary size) is: {model.cum_table.shape}')
  #print(model.cum_table)
  sen_vectors = get_sentence_vectors(model, tokenized_items)
  return sen_vectors

In [15]:
X_train = vectorize_by_wor2vec(X_train)
X_val  = vectorize_by_wor2vec(X_val)

shape of word2vec model (vocabulary size) is: (228740,)
shape of word2vec model (vocabulary size) is: (68795,)


# Neural Network

In [16]:
mlp_clf = MLPClassifier(solver='sgd',
                    alpha=1e-5,
                    hidden_layer_sizes=(10,),
                    random_state=1)
mlp_clf.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(10,), random_state=1,
              solver='sgd')

In [17]:
train_pred = mlp_clf.predict(X_train)
val_pred = mlp_clf.predict(X_val)

# Report metrics like precision, recall, F1-score.

In [52]:
from sklearn.metrics import accuracy_score
print("Accuracy of training set is : ", accuracy_score(y_train, train_pred))

print("Accuracy of validation set is: ", accuracy_score(y_val, val_pred))

Accuracy of training set is :  0.781115987251581
Accuracy of validation set is:  0.7763056899716336


In [34]:
from sklearn.metrics import precision_score
precision = precision_score(y_train,train_pred, pos_label= 1, average= 'weighted')
precision1 = precision_score(y_val,val_pred, pos_label= 1, average= 'weighted')
print('Precision value on training data is :%.3f\n' % precision)
print('Precision value on validation data is :%.3f\n' % precision1)

Precision value on training data is :0.719

Precision value on validation data is :0.625



In [51]:
from sklearn.metrics import recall_score
recall = recall_score(y_train,train_pred,average= 'micro')
recall1 = recall_score(y_val,val_pred,average= 'micro')
print('Recall value on training data is : %.3f\n' % recall)
print('Recall value on validation data is : %.3f\n' % recall1)

Recall value on training data is : 0.781

Recall value on validation data is : 0.776



In [58]:
from sklearn.metrics import f1_score
f1 = f1_score(y_train,train_pred, pos_label= '1',average= 'weighted')
f1_val = f1_score(y_val,val_pred, pos_label= '1',average= 'weighted')
print('F1 score of training data is : %.3f\n' % f1)
print('F1 score of validation data is : %.3f\n' % f1_val)

F1 score of training data is : 0.714

F1 score of validation data is : 0.685

